# Hospital Encounter Analysis by Alyssa 
Data provided by ELVTR course Data Analysis in Healthcare

## Data loading and preprocessing
Let's start with loading and checking for duplicates, and missing data.

In [9]:
#import pandas the data analysis library for python
import pandas as pd 

#upload excel file
encdf = pd.read_excel("hospital_encounter_data_10_24.xlsx")

#upload 2nd tab 
drgdf = pd.read_excel("hospital_encounter_data_10_24.xlsx", sheet_name="DRGTable")

print(encdf.head())

   encounter_id  admission_date origin  service_line      loc    drg  \
0  5AP7T95R6HG6           44927    ED   Internal Med  GEN MED  392.0   
1  UBE6BNHXJDV6           44927    ED   Internal Med  GEN MED  871.0   
2  2G6U4ZHPWW5V           44927    ED   Internal Med  GEN MED  871.0   
3  BSHAPPAB3YFC           44927    ED   Internal Med      PCU  871.0   
4  V6XH3DEFDTTT           44927    ED   Internal Med  GEN MED  871.0   

   discharge_date  patient_zipcode  
0           44939            65072  
1           44931            64742  
2           44929            66524  
3           44957            64150  
4           44930            64097  


### Check for missing values

In [11]:
print( encdf.isna().any() )


encounter_id       False
admission_date     False
origin             False
service_line       False
loc                 True
drg                 True
discharge_date     False
patient_zipcode    False
dtype: bool


In [17]:
encdf[encdf['loc'].isna()]

,encounter_id,admission_date,origin,service_line,loc,drg,discharge_date,patient_zipcode
6,79TGKHP4BTKK,44927,ED,Internal Med,NaN,NaN,44931,63546
7,K5XEENF2JMEZ,44927,ED,Internal Med,NaN,291.0,44931,50853
58,6QU7N2PX65AD,44927,Direct,Internal Med,NaN,291.0,44931,66429
62,GA2WXYSFZKNY,44934,ED,Internal Med,NaN,871.0,44934,65286
110,KF9BMPEHKU3D,44934,Direct,Internal Med,NaN,871.0,44944,64184
...,...,...,...,...,...,...,...,...
97212,8PJUN96RVP74,44667,OR,General Surg,NaN,621.0,44667,50251
97283,2N4XZUBPP7RJ,44730,ED,General Surg,NaN,621.0,44730,64457
97310,H4E5T5GNF2QZ,44751,OR,General Surg,NaN,331.0,44757,50950
97315,QNEPJ8JU8NN2,44758,ED,General Surg,NaN,621.0,44758,50840
